# Introduction
In this notes I try to explain how maxvol based skeleton approximation works for matrices and tensors.
I mostly use the following sources:
https://oseledets.github.io/news/cross-approximation-intro/
http://nbviewer.ipython.org/github/oseledets/nla2015/blob/b6af104bbe25e75be5c7314ad15eed3023752f53/lectures/lecture-15.ipynb
https://advancedoptimizationatharvard.wordpress.com/2014/04/06/skeleton-decomposition/

# Matrix case
Suppose that an $n \times m$ matrix $A$ has rank $r$. Then it can be exactly recovered from $r$ columns and $r$ rows as
$$
A = C \hat{A}^{-1}R,
$$
where $C$ are $r$ columns of $A$, $R$ are $r$ rows of $A$ and $\hat{A}$ is a submatrix on their intersection.
That means, that only $(n+m)r$ elements are required.
What about the stability of this decomposition?
Suppose $A$ can be approximated with low rank only approximately:
$$
A=A_r+E,
$$
with $||E||=\varepsilon$.
Now the quality of approximation depends on the choice of the submatrix $\hat{A}$. A good choice for
 the submatrix is the maximum volume: among all $r\times r$ submatrices select the one that has largest volume, i.e. the maximum absolute value of the determinant.
Then, the corresponding approximation
$$
||A - A_{\text{maxvol}}||_C \leq (r+1) \sigma_{r+1},
$$
where $\sigma_{r+1}$ is the $(r+1)$-th singular value of A and the error is measured in the Chebyshev norm (maximum absolute value).

<img src="figures/skeletondecomposition.jpg">


# Finding the maximum volume submatrix
The problem of finding the maximum volume submatrix is very hard in theory, but greedy algorithms work suprisingly well.
## $n \times r$ matrix
Lets start with a simplified problem: given an $n \times r$ matrix $A$ find $r$ rows corresponding to the maximum volume submatrix.

We will use an iterative algorithm: it starts from a non-singular $r \times r$ submatrix and then substitutes one row at a time (greedy approach).
For this problem, the convergence is very fast. It is possible to avoid explicitly computing determinants and make the iteration time small (see details in the last section).

An efficient implementations of the maxvol algorithm are available both [in MATLAB](https://github.com/oseledets/TT-Toolbox) and [in Python](https://github.com/oseledets/ttpy/tree/master/tt).

## $n \times m$ matrix



## Speeding up iterations
The reason why this greedy algorithm works so well is that you can implement each iteration efficiently, without actually computing determinants.

This part is not that important and you can safely omit it.

Start with some non-singular submatrix $\widehat{A}$. Lets assume that we've chosen the first $r$ rows of $A$. Lets form a matrix $B = A \widehat{A}^{-1}$:
$$
B = \begin{pmatrix}
I\\ 
Z
\end{pmatrix}
$$
Now that lets assume that we want to swap row $i <= r$ with row $j > r$ and obtain the new $B$ which we will call $B^{i,j}$.
What is the new determinant?

First of all, $|C D| = |C| |D|$, so the determinant of the new $\widehat{A}_{\text{new}}$ equals to the determinant of the first $r$ rows in $B^{i,j}$ times the determinant of old $\widehat{A}_{\text{old}}$. Consider an example with $r = 5$, $i = 3$ and 
$$
\underbrace{\left | \begin{pmatrix}
A_{1, 1} & A_{1, 2} & A_{1, 3} & A_{1, 4} & A_{1, 5}\\ 
A_{2, 1} & A_{2, 2} & A_{2, 3} & A_{2, 4} & A_{2, 5}\\ 
A_{j, 1} & A_{j, 2} & A_{j, 3} & A_{j, 4} & A_{j, 5}\\ 
A_{4, 1} & A_{4, 2} & A_{4, 3} & A_{4, 4} & A_{4, 5}\\ 
A_{5, 1} & A_{5, 2} & A_{5, 3} & A_{5, 4} & A_{5, 5}\\ 
\end{pmatrix}  \right |}_{\text{$\widehat{A}_{\text{new}}$}}
=
\left |\begin{pmatrix}
1 & 0 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0 & 0\\ 
B_{j,1} & B_{j,2} & B_{j,3} & B_{j,4} & B_{j,5}\\ 
0 & 0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 0 & 1
\end{pmatrix}  \right |
\underbrace{\left | \begin{pmatrix}
A_{1, 1} & A_{1, 2} & A_{1, 3} & A_{1, 4} & A_{1, 5}\\ 
A_{2, 1} & A_{2, 2} & A_{2, 3} & A_{2, 4} & A_{2, 5}\\ 
A_{3, 1} & A_{3, 2} & A_{3, 3} & A_{3, 4} & A_{3, 5}\\ 
A_{4, 1} & A_{4, 2} & A_{4, 3} & A_{4, 4} & A_{4, 5}\\ 
A_{5, 1} & A_{5, 2} & A_{5, 3} & A_{5, 4} & A_{5, 5}\\ 
\end{pmatrix}  \right |}_{\text{$\widehat{A}_{\text{old}}$, doesn't depend on $i$ and $j$}}
$$

Since the determinant of $\widehat{A}_{\text{old}}$ is fixed, we should choose the pair $(i, j)$ that maximazes the determinant of the first $r$ rows in $B^{i,j}$.

Lets return to the example and use [the Laplace's formula](https://en.wikipedia.org/wiki/Determinant#Laplace.27s_formula_and_the_adjugate_matrix) for the $i$-th row:
$$
\left |\begin{pmatrix}
1 & 0 & 0 & 0 & 0\\ 
0 & 1 & 0 & 0 & 0\\ 
B_{j,1} & B_{j,2} & B_{j,3} & B_{j,4} & B_{j,5}\\ 
0 & 0 & 0 & 1 & 0\\ 
0 & 0 & 0 & 0 & 1
\end{pmatrix}  \right | = (-1)^{(3 + 1)} B_{j,1} \underbrace{\left |\begin{pmatrix}
 0 & 0 & 0 & 0\\ 
 1 & 0 & 0 & 0\\ 
 0 & 0 & 1 & 0\\ 
 0 & 0 & 0 & 1
\end{pmatrix}  \right |}_{0} + (-1)^{(3 + 2)} B_{j,2} \cdot 0 +(-1)^{(3 + 3)} B_{j,3}\underbrace{\left |\begin{pmatrix}
 1 & 0 & 0 & 0\\ 
 0 & 1 & 0 & 0\\ 
 0 & 0 & 1 & 0\\ 
 0 & 0 & 0 & 1
\end{pmatrix}  \right |}_{1} + (-1)^{(3 + 4)} B_{j,4} \cdot 0 + (-1)^{(3 + 5)}B_{j,5} \cdot 0 = B_{j,3}
$$

In general, the determinant of the first $r$ rows of $B$ after swaping $i$-th and $j$-th rows is just the $(i, j)$ element: $|B^{i,j}| = B_{j, i}$.

So to find the pair $(i, j)$ that maximizes the absolute value of the determinant we just have to find $(i, j) = arg\max_{i, j} |B_{i j}|$.

The algorithm:
1. Chose some non-singular $r\times r$ submatrix $\widehat{A}_{\text{old}}$ of A;
2. Compute $B = A \widehat{A}_{\text{old}}^{-1}$;
$$
B = \begin{pmatrix}
I\\ 
Z
\end{pmatrix}
$$
3. Find the maximum absolute value element in $B$: $(i, j) = arg\max_{i, j} |B_{i j}|$;
4. Stop if maximal element is less than $(1+\delta)$;
5. Swap $i$-th row with $j$-th row;
6. Update the matrix $B$ so it again would be equal to $B = A \widehat{A}_{\text{new}}^{-1}$.
7. Goto 3.

The only piece missing is how to efficiently update matrix $B$ after swapping two rows. To do it we have to recalculate $\widehat{A}_{\text{new}}^{-1}$, which is easy, since $\widehat{A}_{\text{new}}$ differs from $\widehat{A}_{\text{old}}$ only in one row ($i$-th), so we can use [Woodbury identity](https://en.wikipedia.org/wiki/Woodbury_matrix_identity) to update the inverse matrix.

In [16]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import tt
from tt.maxvol import maxvol
n = 256
r = 10
a = np.random.randn(*(n, r))
ind = np.sort(maxvol(a, nswp=1))
a_hat = A[ind, :]
err = np.linalg.norm(a - a.dot(np.linalg.inv(a_hat)).dot(a_hat))
print err

2.22729951264e-13


In [14]:
for r in [1, 5, 10, 15]:
    u1 = u[:, :r]
    v1 = v[:r, :]
    v1 = v1.T
    s1 = s[:r]
    
    """ Computing maxvol indices """
    indu = np.sort(maxvol(u1.dot(np.diag(s))))
    indv = np.sort(maxvol(v1))
    C = a[:, indv]
    C = np.linalg.solve(C[indu, :].T, C.T).T # Computing CC^{-1}
    R = a[indu, :]
    
    er_skel = np.linalg.norm(a - np.dot(C, R))
    
    print 'SVD er:', er_svd, 'Skeleton er:', er_skel

ValueError: shapes (256,1) and (10,10) not aligned: 1 (dim 1) != 10 (dim 0)